In [1]:
import numpy as np
from numpy.linalg import inv
from astropy.io import fits
import os
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import textwrap

In [2]:
#import New_DRRP_Functions as DRRP
from New_DRRP_Functions import *

In [3]:
# Theta from 0 to 180 degrees (for the first QWP). The second QWP rotates at 5 times this rate (25 degree increments)
theta = np.linspace(0, np.pi, 37) # 5 degree increments for the first QWP
longtheta = np.linspace(0, np.pi, 46) # 4 degree increments (this is preferred)
degtheta = theta*180/np.pi

In [4]:
# Define the identity matrix and other matrices which are useful for the Mueller calculus
M_identity = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
A = np.array([1, 0, 0, 0])
B = np.array([[1], [0], [0], [0]])

In [5]:
# Mueller matrix for a linear polarizer, with angle a between transmission axis and horizontal (radians)
def linear_polarizer(a):
    M01 = np.cos(2*a)
    M02 = np.sin(2*a)
    M10 = np.cos(2*a)
    M11 = np.cos(2*a)**2
    M12 = np.cos(2*a)*np.sin(2*a)
    M20 = np.sin(2*a)
    M21 = np.cos(2*a)*np.sin(2*a)
    M22 = np.sin(2*a)**2

    return 0.5*np.array([[1, M01, M02, 0], 
                         [M10, M11, M12, 0], 
                         [M20, M21, M22, 0], 
                         [0, 0, 0, 0]])

In [6]:
# Mueller matrix for a linear retarder (waveplate). Angle of fast axis a, retardance r in radians
def linear_retarder(a, r):
    M11 = np.cos(2*a)**2 + np.cos(r)*np.sin(2*a)**2
    M12 = np.cos(2*a)*np.sin(2*a)*(1-np.cos(r))
    M13 = -np.sin(2*a)*np.sin(r)
    M21 = M12
    M22 = np.sin(2*a)**2 + np.cos(2*a)**2*np.cos(r)
    M23 = np.cos(2*a)*np.sin(r)
    M31 = -M13
    M32 = -M23
    M33 = np.cos(r)

    return np.array([[1, 0, 0, 0], 
                     [0, M11, M12, M13], 
                     [0, M21, M22, M23], 
                     [0, M31, M32, M33]])

In [7]:
# Sorting function for extracting filenames based on last number in the filename (the angle of rotation)
def extract_number(filename):
    match = re.findall(r'\d+(?:\.\d+)?', filename)
    if match:
        return float(match[-1])

In [8]:
# Gives the condition number of eventual Mueller matrix (made by Jaren)
def condition_number(matrix):
    minv = np.linalg.pinv(matrix)

    # Compute maximum norm
    norm = np.linalg.norm(matrix, ord=np.inf)
    ninv = np.linalg.norm(minv, ord=np.inf)

    return norm*ninv

In [4]:
# Get intensity values from each spot in the reduced images. reduced_filename should just be the start of the name (leave out the last number, the angle). 
def extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff=5000):
    I_left = np.array([])
    I_right = np.array([])
    bad_indices = np.array([])
    longtheta = np.linspace(0, np.pi, 46)

    for filename in sorted(os.listdir(reduced_folder), key = extract_number):
        if filename.startswith(reduced_filename):
            with fits.open(os.path.join(reduced_folder, filename)) as hdul:
                reduced_img_data = hdul[0].data
                ys, xs, = np.indices(reduced_img_data.shape)
                lradius = np.sqrt((ys-lcenter[0])**2+(xs-lcenter[1])**2)
                rradius = np.sqrt((ys-rcenter[0])**2+(xs-rcenter[1])**2)

                lbackground_mask = (lradius > 20) & (lradius < 26)
                rbackground_mask = (rradius > 20) & (rradius < 26)   # Index the background around each spot, take the median value

                background_lmedian = np.median(reduced_img_data[lbackground_mask])
                background_rmedian = np.median(reduced_img_data[rbackground_mask])

                lflux = np.sum(reduced_img_data[lradius < maxradius] - background_lmedian)   # Now take the flux with the background mask subtracted
                rflux = np.sum(reduced_img_data[rradius < maxradius] - background_rmedian)
                I_left = np.append(I_left, lflux)
                I_right = np.append(I_right, rflux)

                if lflux+rflux < cutoff:
                    print("Warning: low flux detected, check the image " + filename + ", index: " + str(sorted(os.listdir(reduced_folder), key = extract_number).index(filename)))
                    bad_indices = np.append(bad_indices, sorted(os.listdir(reduced_folder), key = extract_number).index(filename))
                else:
                    continue 

    # Makes the array a list of integers that can be used to index the other array
    bad_indices = bad_indices.astype(int)
    # Deletes the bad indices from the data
    I_left = np.delete(I_left, bad_indices)
    I_right = np.delete(I_right, bad_indices)
    new_angles = np.delete(longtheta, bad_indices)

    return I_left, I_right, new_angles, bad_indices

In [5]:
# Original function to compute the Mueller matrix of a sample based on DRRP intensity measurements
def original_full_mueller_polarimetry(thetas, I_meas=1, LPA_angle=0, return_condition_number=False, M_in=None):
    nmeas = len(thetas)
    Wmat = np.zeros([nmeas, 16])
    Pmat = np.zeros([nmeas])
    th = thetas

    for i in range(nmeas):
        # Mueller Matrix of generator (linear polarizer and a quarter wave plate)
        Mg = linear_retarder(th[i], np.pi/2) @ linear_polarizer(0)

        # Mueller Matrix of analyzer (one channel of the Wollaston prism is treated as a linear polarizer. The right spot is horizontal (0) and the left spot is vertical(pi/2))
        Ma = linear_polarizer(LPA_angle) @ linear_retarder(th[i]*5, np.pi/2)

        # Data reduction matrix. Taking the 0 index ensures that intensity is the output
        Wmat[i,:] = np.kron(Ma[0,:], Mg[:,0])

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat[i] = (Ma[0,:] @ M_in @ Mg[:,0]) * I_meas
        else:
            Pmat[i] = I_meas[i]

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M = np.linalg.pinv(Wmat) @ Pmat
    M = np.reshape(M,[4,4])

    if return_condition_number == True:
        return M, condition_number(Wmat)
    else:
        return M

In [6]:
# Modified function to compute the Mueller matrix of a sample based on DRRP intensity measurements
# Needs both channel data at the same time to measure Q
def modified_full_mueller_polarimetry(thetas, I_minus, I_plus, return_condition_number=False, M_in=None):
    nmeas = len(thetas)  # Number of measurements
    Wmat = np.zeros([nmeas, 16])
    Pmat = np.zeros([nmeas])
    th = thetas
    Q = I_plus - I_minus   # Difference in intensities measured by the detector. Plus should be the right spot, minus the left spot
    I_total = I_plus + I_minus

    for i in range(nmeas):
        # Mueller Matrix of generator (linear polarizer and a quarter wave plate)
        Mg = linear_retarder(th[i], np.pi/2) @ linear_polarizer(0)

        # Mueller Matrix of analyzer (one channel of the Wollaston prism is treated as a linear polarizer. The right spot is horizontal (0) and the left spot is vertical(pi/2))
        #Ma = (linear_polarizer(0)-linear_polarizer(np.pi/2)) @ linear_retarder(th[i]*5, np.pi/2)
        Ma = linear_retarder(th[i]*5, np.pi/2)  # take out the wollaston prism part, because we are essentially measuring Q directly after the quarter wave plate

        # Ma and Mg are both 4x4 matrices, so the output is a 16x1 vector
        Wmat[i,:] = np.kron((Ma)[1,:], Mg[:,0])
        #Wmat[i,:] = np.kron(Ma_plus[1,:], Mg[:,1]) - np.kron(Ma_minus[1,:], Mg[:,1])

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat[i] = (Ma[1,:] @ M_in @ Mg[:,0])
            #Pmat[i] = ((Ma_plus[1,:] @ M_in @ Mg[:,1]) - (Ma_minus[1,:] @ M_in @ Mg[:,1]))*1
        else:
            Pmat[i] = Q[i]  #Pmat is essentially what we are measuring, the normailzed Q. Maybe Q[i]/I_total[i]

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M = np.linalg.pinv(Wmat) @ Pmat
    M = np.reshape(M,[4,4])

    if return_condition_number == True:
        return M, condition_number(Wmat)
    else:
        return M

In [7]:
# try to make a version using total intensity that just gives the top row of the Mueller matrix
def top_modified_full_mueller_polarimetry(thetas, I_minus, I_plus, return_condition_number=False, M_in=None):
    nmeas = len(thetas)  # Number of measurements
    Wmat = np.zeros([nmeas, 16])
    Pmat = np.zeros([nmeas])
    th = thetas
    Q = I_plus - I_minus   # Difference in intensities measured by the detector. Plus should be the right spot, minus the left spot
    I_total = I_plus + I_minus

    for i in range(nmeas):
        # Mueller Matrix of generator (linear polarizer and a quarter wave plate)
        Mg = linear_retarder(th[i], np.pi/2) @ linear_polarizer(0)

        # Mueller Matrix of analyzer (one channel of the Wollaston prism is treated as a linear polarizer. The right spot is horizontal (0) and the left spot is vertical(pi/2))
        #Ma = (linear_polarizer(0)-linear_polarizer(np.pi/2)) @ linear_retarder(th[i]*5, np.pi/2)
        Ma = linear_retarder(th[i]*5, np.pi/2)  # take out the wollaston prism part, because we are essentially measuring Q directly after the quarter wave plate

        # Ma and Mg are both 4x4 matrices, so the output is a 16x1 vector
        Wmat[i,:] = np.kron((Ma)[0,:], Mg[:,0])
        #Wmat[i,:] = np.kron(Ma_plus[1,:], Mg[:,1]) - np.kron(Ma_minus[1,:], Mg[:,1])

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat[i] = (Ma[0,:] @ M_in @ Mg[:,0])
            #Pmat[i] = ((Ma_plus[1,:] @ M_in @ Mg[:,1]) - (Ma_minus[1,:] @ M_in @ Mg[:,1]))*1
        else:
            Pmat[i] = I_total[i]  #Pmat is essentially what we are measuring, the normailzed Q

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M = np.linalg.pinv(Wmat) @ Pmat
    M = np.reshape(M,[4,4])

    if return_condition_number == True:
        return M, condition_number(Wmat)
    else:
        return M

In [47]:
# Returns the sum of two matrices, one calculated using I to get the top row, and the other using Q to get the bottom 3 rows
def combined_full_mueller_polarimetry(thetas, I_minus, I_plus, M_in=None):
    nmeas = len(thetas)  # Number of measurements
    Wmat1 = np.zeros([nmeas, 16])
    Pmat1 = np.zeros([nmeas])
    Wmat2 = np.zeros([nmeas, 16])
    Pmat2 = np.zeros([nmeas])
    th = thetas
    Q = I_plus - I_minus   # Difference in intensities measured by the detector. Plus should be the right spot, minus the left spot
    I_total = I_plus + I_minus

    for i in range(nmeas):
        Mg = linear_retarder(th[i], np.pi/2) @ linear_polarizer(0) # Mueller matrix of generator (linear polarizer and quarter wave plate)

        # The right spot is horizontal (angle=0, I plus) and the left spot is vertical (angle=pi/2, I minus)
        Ma = linear_retarder(th[i]*5, np.pi/2)  # Mueller matrix of analyzer (quarter wave plate)

        # Ma and Mg are both 4x4 matrices, so the output is a 16x1 vector
        Wmat1[i,:] = np.kron((Ma)[0,:], Mg[:,0]) # for the top row, using intensities
        Wmat2[i,:] = np.kron((Ma)[1,:], Mg[:,0]) # for the bottom 3 rows, using Q

        # M_in is some example Mueller matrix. Providing this input will test theoretical Mueller matrix. Otherwise, the raw data is used
        if M_in is not None:
            Pmat1[i] = (Ma[0,:] @ M_in @ Mg[:,0])
            Pmat2[i] = (Ma[1,:] @ M_in @ Mg[:,0])
        else:
            Pmat1[i] = I_total[i]  #Pmat is a vector of measurements (either I or Q)
            Pmat2[i] = Q[i] # Divide by I?

    # Compute Mueller matrix using Moore-Penrose pseudo invervse
    M1 = np.linalg.pinv(Wmat1) @ Pmat1
    M1 = np.reshape(M1, [4,4])

    M2 = np.linalg.pinv(Wmat2) @ Pmat2
    M2 = np.reshape(M2, [4,4])

    M = np.zeros([4,4])
    M[0,:] = M1[0,:]
    M[1:4,:] = M2[1:4,:]

    return M

Note the difference between defining filepath in Windows (top) vs Linux (bottom)

In [10]:
reduced_filename = 'Reduced_DRRP_'
#reduced_folder = r"C:\\Users\\EPL User\\Desktop\\L_Plate_Characterization\\Calibration\\Calibration_Reduced\\Cal_1600_Reduced\\"
reduced_folder = "/home/shared/exoserver/Lab_Data/Mueller_Matrix_Polarimeter/L_Plate_Characterization/Calibration/Calibration_Reduced/Cal_1600_Reduced/"
lcenter = [258, 255]
rcenter = [258, 332]
maxradius = 10
cutoff = 5000

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Cal_Il_1600 = extracted_data[0]
Cal_Ir_1600 = extracted_data[1]
Cal_theta1600 = extracted_data[2]

In [40]:
M1 = original_full_mueller_polarimetry(Cal_theta1600, Cal_Ir_1600, 0)
M1 = M1/np.max(M1)
print(M1)
M2 = original_full_mueller_polarimetry(Cal_theta1600, Cal_Il_1600, np.pi/2)
M2 = M2/np.max(M2)
print(M2)
print(np.arccos(M1[3,3]))

[[ 1.          0.05152693  0.20517248  0.00178048]
 [-0.01048165  0.92522473 -0.48160509  0.00187048]
 [-0.03737062  0.4777589   0.91871019  0.00526699]
 [ 0.0040122  -0.00341311 -0.00120618  0.98459564]]
[[ 1.00000000e+00 -5.00787093e-02 -1.96260741e-01  1.77788613e-04]
 [-5.58379564e-03  9.06623893e-01 -4.55664149e-01  1.16617389e-03]
 [-3.88739259e-02  4.59614971e-01  8.82645393e-01 -2.14018921e-03]
 [-7.49722616e-04  1.34843279e-03 -2.76461362e-03  9.61475066e-01]]
0.1757502176527439


In [15]:
M_ex = np.array([[1, 2, 3, 4], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]])
print(M_ex)
np.kron(M_ex[1,:], M_ex[:,0])

[[1 2 3 4]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]]


array([2, 4, 6, 8, 2, 4, 6, 8, 2, 4, 6, 8, 2, 4, 6, 8])

In [13]:
theta_ex = np.linspace(0, np.pi, 37)
Cal_Il_1600_ex = np.linspace(0, np.pi, 37)
Cal_Ir_1600_ex = np.linspace(0, np.pi, 37)

In [13]:
# Version with (M+ - M-). It doesn't seem to make a difference
# Works for the top row but not the others
M6 = modified_full_mueller_polarimetry(theta_ex, Cal_Il_1600_ex, Cal_Ir_1600_ex, M_in=M_ex)
print(M6)

[[ 0.00000000e+00 -9.41602747e-16 -8.64861082e-16 -2.08098438e-15]
 [ 2.00000000e+00  2.00000000e+00  2.00000000e+00  2.00000000e+00]
 [ 3.00000000e+00  3.00000000e+00  3.00000000e+00  3.00000000e+00]
 [ 4.00000000e+00  4.00000000e+00  4.00000000e+00  4.00000000e+00]]


In [22]:
M7 = top_modified_full_mueller_polarimetry(theta_ex, Cal_Il_1600_ex, Cal_Ir_1600_ex, M_in=M_ex)
print(M7)

[[1. 2. 3. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [33]:
M8 = combined_full_mueller_polarimetry(theta_ex, Cal_Il_1600_ex, Cal_Ir_1600_ex, M_in=M_ex)
print(M8)

[[1. 2. 3. 4.]
 [2. 2. 2. 2.]
 [3. 3. 3. 3.]
 [4. 4. 4. 4.]]


In [38]:
print(combined_full_mueller_polarimetry(theta_ex, Cal_Il_1600_ex, Cal_Ir_1600_ex, M_in=M2))

[[ 1.00000000e+00 -5.00787093e-02 -1.96260741e-01  1.77788613e-04]
 [-5.58379564e-03  9.06623893e-01 -4.55664149e-01  1.16617389e-03]
 [-3.88739259e-02  4.59614971e-01  8.82645393e-01 -2.14018921e-03]
 [-7.49722616e-04  1.34843279e-03 -2.76461362e-03  9.61475066e-01]]


In [48]:
M9 = combined_full_mueller_polarimetry(Cal_theta1600, Cal_Il_1600, Cal_Ir_1600)
M9 = M9/np.max(M9)
print(M9)

[[ 1.          0.00148209 -0.0019202   0.00117834]
 [-0.01612105  0.97855426 -0.20289116  0.00301233]
 [-0.03368077  0.44931743  0.89274259  0.0037962 ]
 [ 0.00386122 -0.01093588 -0.00640964  0.97519054]]


In [46]:
print(M9-M2)

[[ 0.          0.0515608   0.19434054  0.00100056]
 [-0.01053725  0.07193037  0.25277298  0.00184616]
 [ 0.00519315 -0.01029754  0.01009719  0.00593639]
 [ 0.00461094 -0.01228432 -0.00364503  0.01371548]]
